In [1]:
from helper import *
df = create_df_from_clipp("sample_snv.txt", "sample_cna.txt", "sample_purity.txt")

In [2]:
df

,mutation_chrom,mutation,position,region,ref_counts,alt_counts,major_cn,minor_cn,normal_cn,tumour_purity
0,1,2,2,1,373,124,1,1,2,0.9
1,1,7,7,1,369,135,1,1,2,0.9
2,1,12,12,1,381,144,1,1,2,0.9
3,1,17,17,1,369,114,1,1,2,0.9
4,1,22,22,1,388,130,1,1,2,0.9
...,...,...,...,...,...,...,...,...,...,...
482,1,2412,2412,1,565,235,2,1,2,0.9
483,1,2417,2417,1,274,216,1,1,2,0.9
484,1,2422,2422,1,514,237,2,1,2,0.9
485,1,2427,2427,1,275,208,1,1,2,0.9


In [8]:
No_mutations = np.shape(df)[0]

In [ ]:
cn_tmp = df['position', 'major_cn', 	'minor_cn',	'normal_cn']
cn_tmp = cn_tmp[cn_tmp['minor_cn'].notna()]
No_cnLines = cn_tmp.shape[0]
mutation_chrom = df['mutation_chrom']
mutation_pos = df['position']
dropped_SNV = []
mut_cna_id = np.array([-1] * No_mutations)
for x in range(No_mutations):
    for i in range(No_cnLines):
        if mutation_chrom.iloc[x] == cn_tmp.iloc[i, 0] and cn_tmp.iloc[i, 1] <= mutation_pos.iloc[x] <= cn_tmp.iloc[i, 2]:
            mut_cna_id[x] = i
            break

valid_ind = mut_cna_id > 0
drop_ind = ~valid_ind
dropped_SNV.extend(CombineReasons(mutation_chrom, mutation_pos, drop_ind, "The SNV does not have valid copy number."))
if valid_ind.sum() < VALID_CONT:
    raise ValueError(f'The sample with SNV {snv_file} has less than {VALID_CONT} SNVs that have valid copy number status.')

mutation_chrom = mutation_chrom[valid_ind]
mutation_pos = mutation_pos[valid_ind]
minor_read = minor_read[valid_ind]
total_read = total_read[valid_ind]
No_mutations = valid_ind.sum()
mut_cna_id = mut_cna_id[valid_ind]
minor_copy_lim = cn_tmp.iloc[mut_cna_id, [cn_tmp.columns.get_loc("minor_cn"), cn_tmp.columns.get_loc("major_cn")]].max(axis=1)
total_count = cn_tmp.iloc[mut_cna_id, cn_tmp.columns.get_loc("total_cn")]


In [6]:
mutation_pos = df['position']
minor_read = df['alt_counts']
total_read = df['ref_counts'] + df['alt_counts']
minor_count = df['minor_cn']
total_count = df['major_cn'] + df['minor_cn']
purity = df['tumour_purity']
multiplicity = np.round(minor_read / total_read / purity * (total_count * purity + (1 - purity) * 2))
minor_count = np.minimum(minor_count, multiplicity)
mut_cna_id = np.array([-1] * np.shape(df)[0])

minor_copy_lim = df.iloc[mut_cna_id, [df.columns.get_loc("minor_cn"), df.columns.get_loc("major_cn")]].max(axis=1)
phi = 2 / (minor_count / (minor_read / total_read) - total_count + 2)
valid_ind = (phi <= 1.5) & (phi > 0)
clonal_ind = phi > 1.5
if clonal_ind.sum() > 0:
    outlier_higherEnd = np.column_stack((df[clonal_ind], mutation_pos[clonal_ind], total_count[clonal_ind], minor_count[clonal_ind]))
drop_ind = ~valid_ind


In [7]:
sum(drop_ind)

20

In [ ]:

minor_count = np.minimum(minor_copy_lim, multiplicity)
minor_count[minor_count == 0] = 1

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
!docker run -v $(pwd):/Sample clipp python3 /CliPP/run_clipp_main.py -i /Sample/test /Sample/sample_snv.txt /Sample/sample_cna.txt /Sample/sample_purity.txt 

Lambda: 0.05	iteration: 7	residual: 0.037493881.143203n: 1	residual: 100
Lambda: 0.175	iteration: 27	residual: 0.02006450.175	iteration: 24	residual: 0.0233678
Lambda: 0.175	iteration: 31	residual: 0.01500660.25	iteration: 22	residual: 0.0320688
Lambda: 0.075	iteration: 33	residual: 0.0333725
Lambda: 0.15	iteration: 27	residual: 0.01883636
Lambda: 0.03	iteration: 18	residual: 0.01700157
Lambda: 0.05	iteration: 22	residual: 0.01869365
Lambda: 0.05	iteration: 26	residual: 0.02093419
Lambda: 0.05	iteration: 32	residual: 0.02027158
Lambda: 0.05	iteration: 33	residual: 0.0196222
Lambda: 0.05	iteration: 43	residual: 0.0110558
Running preprocessing...
Preprocessing finished.
Running the main CliPP function...

Elapsed time: 0.11sec

Main CliPP function finished.


In [ ]:
import pandas as pd

# File path
file_path = '/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/test/final_result/Best_lambda/mutation_assignments_lam0.25.txt'

# Read the data into a DataFrame
df = pd.read_csv(file_path, sep='\t')

# Display the DataFrame
print(df)

     chromosome_index  position  cluster_index
0                   1         2              1
1                   1         7              1
2                   1        12              1
3                   1        17              1
4                   1        22              1
..                ...       ...            ...
476                 1      2412              0
477                 1      2417              0
478                 1      2422              0
479                 1      2427              0
480                 1      2432              0

[481 rows x 3 columns]


In [6]:
ind = [i for i in range(len(df['position']) - 1) if df['position'][i + 1] - df['position'][i] != 5]
df['position'][ind]

82      412
88      447
139     707
154     787
205    1047
252    1287
Name: position, dtype: int64

In [5]:
from snv import * 
from helper import *

In [6]:
df = create_df_from_clipp('sample_snv.txt', 'sample_cna.txt', 'sample_purity.txt')

In [7]:
rho = 0.8
omega = 1
n = df.shape[0]
m = 1
gamma = [0.25]
control_large = 5
res = CliPP2(df, rho, 0.01, omega, n, m)

 10 , 0.00966105356842245

In [8]:
res

{'phi': array([[0.37063343],
        [0.39601155],
        [0.39601155],
        [0.34199891],
        [0.37063343],
        [0.39601155],
        [0.45075181],
        [0.34199891],
        [0.37063343],
        [0.37063343],
        [0.37063343],
        [0.39601155],
        [0.45075181],
        [0.37063343],
        [0.37063343],
        [0.39601155],
        [0.37063343],
        [0.45075181],
        [0.37063343],
        [0.37063343],
        [0.45075181],
        [0.45075181],
        [0.37063343],
        [0.45075181],
        [0.37063343],
        [0.39601155],
        [0.37063343],
        [0.39601155],
        [0.39601155],
        [0.34199891],
        [0.45075181],
        [0.34199891],
        [0.37063343],
        [0.39601155],
        [0.39601155],
        [0.37063343],
        [0.39601155],
        [0.45075181],
        [0.45075181],
        [0.45075181],
        [0.45075181],
        [0.37063343],
        [0.34199891],
        [0.45075181],
        [0.34199891],
   

In [3]:
rho = 0.8
omega = 1
n = df.shape[0]
m = 1
gamma = [0.25]
control_large = 5
ray.shutdown()
ray.init(ignore_reinit_error=True)
res0 = [CliPP2.remote(df, rho, gamma[i], omega, n, m) for i in range(len(gamma))]
res0 = ray.get(res0)
ray.shutdown()
res = res0[find_gamma(res0)]

2025-02-15 20:43:55,338	INFO worker.py:1786 -- Started a local Ray instance.


RayTaskError(UnboundLocalError): [36mray::CliPP2()[39m (pid=5749, ip=127.0.0.1)
  File "/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/snv.py", line 521, in CliPP2
    while k < max_iteration and precision > temp:
                                            ^^^^
UnboundLocalError: cannot access local variable 'temp' where it is not associated with a value

In [4]:
!docker run -v $(pwd):/Sample clipp python3 /CliPP/run_clipp_main.py -i /Sample/test /Sample/sample_snv.txt /Sample/sample_cna.txt /Sample/sample_purity.txt


Lambda: 0.075	iteration: 22	residual: 0.0308924175	iteration: 15	residual: 0.139639
Lambda: 0.225	iteration: 25	residual: 0.0173349125	iteration: 25	residual: 0.050714
Lambda: 0.25	iteration: 18	residual: 0.07480142
Lambda: 0.05	iteration: 37	residual: 0.01353098.15	iteration: 21	residual: 0.0651553
Lambda: 0.03	iteration: 8	residual: 0.022807868
Lambda: 0.15	iteration: 24	residual: 0.0822953
Lambda: 0.25	iteration: 33	residual: 0.0264399
Lambda: 0.03	iteration: 11	residual: 0.0212298
Lambda: 0.1	iteration: 43	residual: 0.01441039
Lambda: 0.1	iteration: 47	residual: 0.01105319
Lambda: 0.03	iteration: 25	residual: 0.0117544
Running preprocessing...
Preprocessing finished.
Running the main CliPP function...

Elapsed time: 0.10sec

Main CliPP function finished.


In [5]:
res

{'phi': array([[0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.67252903],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.67252903],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
        [0.39013311],
   

In [9]:
res0

[{'phi': array([[0.37063343],
         [0.39601155],
         [0.39601155],
         [0.34199891],
         [0.37063343],
         [0.39601155],
         [0.45075181],
         [0.34199891],
         [0.37063343],
         [0.37063343],
         [0.37063343],
         [0.39601155],
         [0.45075181],
         [0.37063343],
         [0.37063343],
         [0.39601155],
         [0.37063343],
         [0.45075181],
         [0.37063343],
         [0.37063343],
         [0.45075181],
         [0.45075181],
         [0.37063343],
         [0.45075181],
         [0.37063343],
         [0.39601155],
         [0.37063343],
         [0.39601155],
         [0.39601155],
         [0.34199891],
         [0.45075181],
         [0.34199891],
         [0.37063343],
         [0.39601155],
         [0.39601155],
         [0.37063343],
         [0.39601155],
         [0.45075181],
         [0.45075181],
         [0.45075181],
         [0.45075181],
         [0.37063343],
         [0.34199891],
    